### Tools

**Models** can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code.
**Tools** are pairing of:

1. A Schema including the name of the tool, a description, and/or argument definations(often a JSON schema).
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [24]:
# GPT MODEL
# model = init_chat_model("gpt-4o-mini")

# GROQ MODEL
model = init_chat_model("groq:llama-3.3-70b-versatile")

# GOOGLE MODEL
# model = init_chat_model(model="google_genai:gemini-flash-latest")

# response = model.stream("How airplanes fly?")

# for chunk in response:
#   print(chunk.text, end=" ", flush=True)

In [25]:
# Tools - Define karna
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
  """Get the weather in a given location"""
  return f"The weather in {location} is sunny"

# Tool ko model ke saath bind karna
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What is the weather in Berani Sindh Pakistan?")
print(response)

tools_calling = response.tool_calls
print(tools_calling)

# if tools_calling:
#   tool_call = tools_calling[0]
#   print(f"\nTool Name: {tool_call['name']}")
#   print(f"Arguments: {tool_call['args']}")
#   print(f"Tool Call ID: {tool_call['id']}")
  
  
#   # Actual tool ko execute karna
#   location = tool_call['args']['location']
#   result = get_weather.invoke({"location": location})
#   print(f"\nTool Result: {result}")
  

content='' additional_kwargs={'tool_calls': [{'id': 'zpq127qty', 'function': {'arguments': '{"location":"Berani Sindh Pakistan"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 225, 'total_tokens': 243, 'completion_time': 0.05762879, 'completion_tokens_details': None, 'prompt_time': 0.013792215, 'prompt_tokens_details': None, 'queue_time': 0.008636809, 'total_time': 0.071421005}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bb1e4-8390-7ab3-b556-689d63bcfb7a-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Berani Sindh Pakistan'}, 'id': 'zpq127qty', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 225, 'output_tokens': 18, 'total_tokens': 243}
[{'name': 'get_weather', 'args': {'location': 'Berani Sindh Pakistan'}, 'id': '

In [26]:
## Tool Execution Loop

messages = [{"role": "user", "content": "What is the weather in Berani Sindh?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)


for tool_call in ai_msg.tool_calls:
  tool_result = get_weather.invoke(tool_call)
  messages.append(tool_result)
  

final_response = model_with_tools.invoke(messages)
print(final_response.text)
  


The weather in Berani Sindh is sunny.


In [27]:
messages

[{'role': 'user', 'content': 'What is the weather in Berani Sindh?'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '4swykkg40', 'function': {'arguments': '{"location":"Berani Sindh"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 224, 'total_tokens': 241, 'completion_time': 0.060183006, 'completion_tokens_details': None, 'prompt_time': 0.01293081, 'prompt_tokens_details': None, 'queue_time': 0.008417154, 'total_time': 0.073113816}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb1e4-99c5-7bf3-91fe-cae41c3e0e99-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Berani Sindh'}, 'id': '4swykkg40', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 224, 'output_tokens': 17, 'total_tokens': 241}),
 ToolMes

In [ ]:
# Tool calls ko dekhna
print("Tool Calls:", response.tool_calls)

# Tool call details
if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nTool Name: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")
    print(f"Tool Call ID: {tool_call['id']}")
    
    # Actual tool ko execute karna
    location = tool_call['args']['location']
    result = get_weather.invoke({"location": location})
    print(f"\nTool Result: {result}")

Tool Calls: [{'name': 'get_weather', 'args': {'location': 'Berlin'}, 'id': '9ac2a610-f189-48dd-ace6-c3584ee4eb6a', 'type': 'tool_call'}]

Tool Name: get_weather
Arguments: {'location': 'Berlin'}
Tool Call ID: 9ac2a610-f189-48dd-ace6-c3584ee4eb6a

Tool Result: The weather in Berlin is sunny


### Multiple Tools Example

In [ ]:
# Multiple tools define karna
from langchain.tools import tool

@tool
def get_temperature(location: str) -> str:
    """Get the temperature in a given location in Celsius"""
    # Dummy data - real mein API call hogi
    temps = {"Berlin": "15", "Paris": "18", "London": "12", "Tokyo": "20"}
    return f"The temperature in {location} is {temps.get(location, '25')}°C"

@tool
def get_humidity(location: str) -> str:
    """Get the humidity level in a given location"""
    return f"The humidity in {location} is 65%"

@tool  
def calculate_sum(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

# Sabhi tools ko bind karna
model_with_multiple_tools = model.bind_tools([get_weather, get_temperature, get_humidity, calculate_sum])

# Test karna
response = model_with_multiple_tools.invoke("What is the temperature and humidity in Berlin?")
print("Response:", response)
print("\nTool Calls:", response.tool_calls)

if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nTool Name: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")
    print(f"Tool Call ID: {tool_call['id']}")
    
    # Actual tool ko execute karna
    location = tool_call['args']['location']
    result = get_humidity.invoke({"location": location})
    print(f"\nTool Result: {result}")

Response: content='' additional_kwargs={'function_call': {'name': 'get_humidity', 'arguments': '{"location": "Berlin"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-preview-09-2025', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019bb1ca-a63e-75a0-aae1-59e18a114aeb-0' tool_calls=[{'name': 'get_temperature', 'args': {'location': 'Berlin'}, 'id': 'ef7e26b2-0547-415c-8cde-477b37cda5dd', 'type': 'tool_call'}, {'name': 'get_humidity', 'args': {'location': 'Berlin'}, 'id': 'f9a82d37-43cd-4dd6-97ca-5e17ab473fe0', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 177, 'output_tokens': 38, 'total_tokens': 215, 'input_token_details': {'cache_read': 0}}

Tool Calls: [{'name': 'get_temperature', 'args': {'location': 'Berlin'}, 'id': 'ef7e26b2-0547-415c-8cde-477b37cda5dd', 'type': 'tool_call'}, {'name': 'get_humidity', 'args': {'location': 'Berlin'}, 'id': 'f9a82d37-43cd-4dd6-97ca-5e17ab473fe0', 'type': 'tool_call'}]



### Complete Tool Execution Flow

In [ ]:
# Tool execution helper function
def execute_tool_calls(response, tools_dict):
    """
    Tool calls ko execute karta hai aur results return karta hai
    """
    if not response.tool_calls:
        return response.content
    
    results = []
    for tool_call in response.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        print(f"\n🔧 Executing Tool: {tool_name}")
        print(f"📝 Arguments: {tool_args}")
        
        # Tool ko execute karna
        if tool_name in tools_dict:
            result = tools_dict[tool_name].invoke(tool_args)
            print(f"✅ Result: {result}")
            results.append({
                'tool': tool_name,
                'args': tool_args,
                'result': result
            })
        else:
            print(f"❌ Tool not found: {tool_name}")
    
    return results

# Tools dictionary banana
tools_dict = {
    'get_weather': get_weather,
    'get_temperature': get_temperature,
    'get_humidity': get_humidity,
    'calculate_sum': calculate_sum
}

# Example 1: Weather query
print("="*50)
print("Example 1: Weather Query")
print("="*50)
response1 = model_with_multiple_tools.invoke("What's the weather in Berlin?")
results1 = execute_tool_calls(response1, tools_dict)

# Example 2: Multiple tools
print("\n" + "="*50)
print("Example 2: Multiple Tools")
print("="*50)
response2 = model_with_multiple_tools.invoke("What is the temperature in Paris and calculate 25 + 37")
results2 = execute_tool_calls(response2, tools_dict)

Example 1: Weather Query

🔧 Executing Tool: get_weather
📝 Arguments: {'location': 'Berlin'}
✅ Result: The weather in Berlin is sunny

Example 2: Multiple Tools

🔧 Executing Tool: get_temperature
📝 Arguments: {'location': 'Paris'}
✅ Result: The temperature in Paris is 18°C

🔧 Executing Tool: calculate_sum
📝 Arguments: {'b': 37, 'a': 25}
✅ Result: 62


### Key Concepts Summary

**Tools kya hain?**
- Functions jo model call kar sakta hai
- External data fetch karne ke liye (APIs, databases, etc.)
- Calculations perform karne ke liye
- Real-world actions execute karne ke liye

**Tool Definition:**
```python
@tool
def function_name(param: type) -> return_type:
    """Description for the model"""
    return result
```

**Important Points:**
1. **Docstring zaruri hai** - Model isko padhkar decide karta hai kab tool use karna hai
2. **Type hints** - Parameters aur return type clear hone chahiye
3. **bind_tools()** - Model ko tools attach karta hai
4. **tool_calls** - Model ki request for tool execution
5. **invoke()** - Actual tool execution

**Flow:**
1. User query → Model
2. Model decides tool chahiye
3. Model returns tool_calls (not actual result)
4. Hum manually tool execute karte hain
5. Result ko model ko wapas bhej sakte hain for final response